In [1]:
import pickle
import MySQLdb
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime
import operator as op
import math
import random
import pandas as pd

In [2]:
# Connect to the MySQL database
def getSQLtest(sql):
    # Connect to the MySQL database
    db = MySQLdb.connect(host='localhost',
                         port=3306,
                         user='root',
                         passwd='root',
                         db='originaldata')
    db_cursor = db.cursor(MySQLdb.cursors.DictCursor)
    sql0 = """
         set sql_mode = 'STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION';
    """       
    db_cursor.execute(sql0)
    db_cursor.execute(sql)
    db_results = db_cursor.fetchall()
    db_cursor.close()
    print ("Fetched Preliminary query results")
    print ("Number of results: " + str(len(db_results)))
    return db_results

In [ ]:
# Preliminary query for feature data
sql1 = """
    SELECT
        al.AliasType,
        al.AliasName,
        
        ve.VenueId,
        ve.VenueSerNum,
        
        ap.AliasSerNum,
        ap.PatientSerNum,
        ap.ScheduledStartTime,
        ap.ScheduledEndTime,
        ap.ActualStartDate,
        ap.ActualEndDate,
        
        ptlmh.ArrivalDateTime,
        ptlmh.HstryDateTime,
        ptlmh.checkedInFlag,
        
        ptl.ArrivalDateTime pltArrivalDateTime,
        ptl.checkedInFlag ,
        
        ap.AppointmentSerNum,
        ap.ScheduledEndTime,
        IF((ap.ActualEndDate IN ('0000-00-00 00:00:00','1970-01-01 00:00:00'))
            OR (DATE(ap.ActualStartDate) != DATE(ap.ActualEndDate)),
        '0', TIMESTAMPDIFF(MINUTE, ap.ActualStartDate, ap.ActualEndDate)) duration,
        TIMESTAMPDIFF(MINUTE, ap.ScheduledStartTime, ap.ScheduledEndTime) allocated,
        co.CourseSerNum,
        pd.DoctorSerNum,
        pl.PlanSerNum,
        IF(pl.TreatmentOrientation = '', 'NULL', pl.TreatmentOrientation) orientation
    FROM
        Appointment ap
    INNER JOIN Diagnosis dx
        ON dx.DiagnosisSerNum = ap.DiagnosisSerNum
    LEFT JOIN DiagnosisTranslation dxt
        ON dxt.DiagnosisCode = dx.DiagnosisCode
    INNER JOIN Patientcopy ptc
        ON ptc.PatientSerNum = ap.PatientSerNum
    INNER JOIN Course co
        ON co.PatientSerNum = ptc.PatientSerNum
    INNER JOIN Plan pl
        ON pl.CourseSerNum = co.CourseSerNum
    INNER JOIN PatientDoctor pd
        ON pd.PatientSerNum = ap.PatientSerNum
    WHERE
        ap.Status LIKE '%%Completed%%'
    AND ap.State = 'Active'
    AND ap.ActualStartDate NOT IN ('0000-00-00 00:00:00','1970-01-01 00:00:00')
    AND pd.PrimaryFlag = 1
    AND pd.OncologistFlag = 1
    AND pl.Status IN ('TreatApproval')
    AND co.CourseSerNum = (
        SELECT co2.CourseSerNum
        FROM Course co2
        INNER JOIN Plan pl2
        ON (    pl2.CourseSerNum = co2.CourseSerNum
            AND pl2.Status IN ('TreatApproval') )
        WHERE
            ap.ActualStartDate  > co2.StartDateTime
        AND co2.CourseId        NOT LIKE '%%QA%%'
        AND co.PatientSerNum    = co2.PatientSerNum
        ORDER BY co2.StartDateTime DESC
        LIMIT 1 )
    AND ap.AliasSerNum IN ('31', '23')
    AND ap.ScheduledStartTime > '2015-01-01 00:00:00'
    GROUP BY
        ptc.PatientSerNum, ap.ScheduledStartTime
    ORDER BY
        ap.AppointmentSerNum ASC
"""